# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
!pip install mysql-connector-python
!pip install selenium
!pip install selenium webdriver-manager

  Using cached mysql_connector_python-9.2.0-cp312-cp312-win_amd64.whl.metadata (6.2 kB)
Using cached mysql_connector_python-9.2.0-cp312-cp312-win_amd64.whl (16.1 MB)
  Using cached selenium-4.29.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached trio-0.29.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
Using cached selenium-4.29.0-py3-none-any.whl (9.5 MB)
Using cached trio-0.29.0-py3-none-any.whl (492 kB)
Using cached trio_websocket-0.12.2-py3-none-any.whl (21 kB)
Using cached attrs-25.1.0-py3-none-any.whl (63 kB)
Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1

In [2]:
# import required library
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import mysql.connector


In [3]:
# Connect to MySQL
def getdbConnection():
    return mysql.connector.connect(
        host="localhost",
        user="root",
        password="Ljx*020907",
        database="subways_db"
    )
connection = getdbConnection()
cursor = connection.cursor()
print("Connected to MySQL database")



Connected to MySQL database


In [4]:
create_table_query = """
CREATE TABLE subway_stores (
id INT AUTO_INCREMENT PRIMARY KEY,
name VARCHAR(255) NOT NULL,
address TEXT,
hours TEXT,
latitude VARCHAR(50),
longitude VARCHAR(50),
waze_link TEXT
);
"""
cursor.execute("USE subways_db")  # Explicitly select the database
cursor.execute(create_table_query)
print(" Table 'subway_stores' created successfully!")


ProgrammingError: 1050 (42S01): Table 'subway_stores' already exists

In [20]:
# Initialize WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Open Subway store locator
url = "https://subway.com.my/find-a-subway"
driver.get(url)

# Wait for the search input field to be available
wait = WebDriverWait(driver, 10)
search_box = wait.until(EC.presence_of_element_located((By.ID, "fp_searchAddress")))

# Enter "Kuala Lumpur" in search bar
search_box.clear()
search_box.send_keys("Kuala Lumpur")

# Click the search button
search_button = wait.until(EC.element_to_be_clickable((By.ID, "fp_searchAddressBtn")))
search_button.click()

# Wait for results to load
wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "fp_listitem")))

stores = []
store_elements = driver.find_elements(By.CLASS_NAME, "fp_listitem")

for store in store_elements:
    try:
        if store.value_of_css_property("display") =="none":
            continue
        # Extract Latitude & Longitude
        latitude = store.get_attribute("data-latitude")
        longitude = store.get_attribute("data-longitude")

        # Extract Store Name
        name = store.find_element(By.CLASS_NAME, "location_left").find_element(By.TAG_NAME, "h4").text

        # Extract Address & Hours (inside infoboxcontent)
        info_box = store.find_element(By.CLASS_NAME, "infoboxcontent")
        p_tags = [p.text.strip() for p in info_box.find_elements(By.TAG_NAME, "p")]
        empty_p_index = p_tags.index("")


        # Extract Address (before the first empty <p>)
        if empty_p_index > 0:
            address = p_tags[0].strip()  # Join if multiple address lines
        else:
            address = ""

        # Extract Operating Hours (after the first empty <p>)
        valid_hours = [h for h in p_tags[empty_p_index + 1:] if h]  # Remove empty values
        hours = ", ".join(valid_hours) if valid_hours else ""

                # Locate the "location_right" div which contains direction links
        location_right = store.find_element(By.CLASS_NAME, "location_right")

        # Find all <a> tags within this div
        direction_links = location_right.find_elements(By.TAG_NAME, "a")

        # Extract the second link (Waze)
        if len(direction_links) > 1:
            waze_link = direction_links[1].get_attribute("href")
        else:
            waze_link = ""

        # Insert into MySQL
        insert_query = """
        INSERT INTO subway_stores (name, address, hours, latitude, longitude, waze_link)
        VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (name, address, hours, latitude, longitude, waze_link))
        connection.commit()

    except Exception as e:
        print(f"Error extracting data: {e}")
        continue

# Close browser
driver.quit()








In [6]:
!pip install fastapi
!pip install uvicorn
!pip install geopy
!pip install nest_asyncio


In [8]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware


In [9]:
nest_asyncio.apply()
# Initialize FastAPI app
app = FastAPI()

# Initialize FastAPI app
app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


@app.get("/outlets")
def get_outlets():
    connection = getdbConnection()
    cursor = connection.cursor(dictionary=True)
    cursor.execute("SELECT * FROM subway_stores")
    outlets = cursor.fetchall()
    cursor.close()
    connection.close()

    if not outlets:
        raise HTTPException(status_code=404, detail="No outlets found")

    return {"outlets": outlets}

uvicorn.run(app, host="127.0.0.1", port=8000)


INFO:     Started server process [8968]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51022 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51022 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51026 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51026 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51026 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:54358 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57485 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57516 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57572 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57606 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57616 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57634 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57641 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57646 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57798 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:57810 - "GET /outlets HTTP/1.1" 200 OK
INFO:     127.0.0.1:58827 - "GET /outlets H

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [8968]
